<h1>Maritime: Collision</h1>

<h4>Selecting relevant rows</h4>

In [ ]:
# --- IGNORE CUS IT WAS RAN ONCE ALREADY --- 
# import pandas as pd
# import ast

# # Load CSV and parse JSON-like fields
# df = pd.read_csv("gfw_encounters_recent.csv",
#                  converters={
#                      'boundingBox': ast.literal_eval,
#                      'regions__mpa': ast.literal_eval,
#                      'regions__eez': ast.literal_eval,
#                      'regions__rfmo': ast.literal_eval,
#                      'regions__fao': ast.literal_eval,
#                      'regions__majorFao': ast.literal_eval,
#                      'regions__eez12Nm': ast.literal_eval,
#                      'regions__highSeas': ast.literal_eval,
#                      'regions__mpaNoTakePartial': ast.literal_eval,
#                      'regions__mpaNoTake': ast.literal_eval
#                  })

# # Select useful columns for collision risk
# risk_cols = [
#     "start", "end",
#     "position__lat", "position__lon", "boundingBox",
#     "distances__startDistanceFromShoreKm", "distances__endDistanceFromShoreKm",
#     "distances__startDistanceFromPortKm", "distances__endDistanceFromPortKm",
#     "encounter__medianDistanceKilometers", "encounter__medianSpeedKnots",
#     "encounter__potentialRisk",
#     "vessel__id", "vessel__ssvid", "vessel__flag", "vessel__type",
#     "encounter__vessel__id", "encounter__vessel__ssvid",
#     "encounter__vessel__flag", "encounter__vessel__type"
# ]

# df_risk = df[risk_cols].copy()

# # Convert times to datetime
# df_risk["start"] = pd.to_datetime(df_risk["start"])
# df_risk["end"] = pd.to_datetime(df_risk["end"])

# # Encounter duration
# df_risk["duration_hours"] = (df_risk["end"] - df_risk["start"]).dt.total_seconds() / 3600

# # Quick filter: potentially high collision risk if distance < 1 km
# df_high_risk = df_risk[df_risk["encounter__medianDistanceKilometers"] < 1]

# print("Cleaned dataset shape:", df_risk.shape)
# print("High-risk encounters (<1km):", len(df_high_risk))
# pd.to_csv(df_risk, "collision/collision_risk.csv", index=False)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

df = pd.read_csv('collision/collision_risk.csv', parse_dates=['start', 'end'])
df.head(5)

,start,end,position__lat,position__lon,boundingBox,distances__startDistanceFromShoreKm,distances__endDistanceFromShoreKm,distances__startDistanceFromPortKm,distances__endDistanceFromPortKm,encounter__medianDistanceKilometers,...,encounter__potentialRisk,vessel__id,vessel__ssvid,vessel__flag,vessel__type,encounter__vessel__id,encounter__vessel__ssvid,encounter__vessel__flag,encounter__vessel__type,duration_hours
0,2025-03-08 06:20:00+00:00,2025-03-21 03:20:00+00:00,19.5021,57.7414,"[57.74136833191366, 19.50214031613387, 57.7413...",3,3,17.914146,17.914146,0.339,...,False,35f7c13c7-7ddc-d6a8-4252-b615bfa6fe24,412549027,CHN,fishing,0c0e88655-5f69-0418-5816-5f48e386114c,412549025,CHN,fishing,309.000000
1,2025-03-08 06:20:00+00:00,2025-03-21 03:20:00+00:00,19.5021,57.7414,"[57.74136833191366, 19.50214031613387, 57.7413...",3,3,17.914146,17.914146,0.339,...,False,0c0e88655-5f69-0418-5816-5f48e386114c,412549025,CHN,fishing,35f7c13c7-7ddc-d6a8-4252-b615bfa6fe24,412549027,CHN,fishing,309.000000
2,2025-03-10 05:30:00+00:00,2025-03-23 18:10:00+00:00,19.4810,57.7305,"[57.73049860026026, 19.480987069680477, 57.730...",2,2,20.042490,20.042490,0.168,...,False,77a0b9ae4-4858-2987-03ee-ed478f1606a0,271072244,TUR,fishing,5338bdfbb-b05d-4925-74d2-4c1513da65df,461000153,OMN,fishing,324.666667
3,2025-03-10 05:30:00+00:00,2025-03-23 18:10:00+00:00,19.4810,57.7305,"[57.73049860026026, 19.480987069680477, 57.730...",2,2,20.042490,20.042490,0.168,...,False,5338bdfbb-b05d-4925-74d2-4c1513da65df,461000153,OMN,fishing,77a0b9ae4-4858-2987-03ee-ed478f1606a0,271072244,TUR,fishing,324.666667
4,2025-03-10 20:00:00+00:00,2025-03-20 18:00:00+00:00,48.3927,-64.4927,"[-64.49272469604246, 48.39265884949554, -64.49...",2,2,39.465820,39.465820,0.054,...,False,60797c0ec-c710-ecba-567d-fa48b79250b4,316004122,CAN,fishing,59b22876a-a3ca-6dce-7e85-b0b3a67f8b49,316008008,CAN,fishing,238.000000


<h4>EDA + Cleaning data</h4>

In [2]:
print("rows, cols:", df.shape)
print("time range:", df['start'].min(), "→", df['end'].max())
print("unique vessels (col vessel__id):", df['vessel__id'].nunique())
print("unique encounter partners:", df['encounter__vessel__id'].nunique())

rows, cols: (8000, 21)
time range: 2025-03-08 06:20:00+00:00 → 2025-04-17 20:00:00+00:00
unique vessels (col vessel__id): 2777
unique encounter partners: 2777


In [3]:
df.isna().sum


<bound method DataFrame.sum of       start    end  position__lat  position__lon  boundingBox  \
0     False  False          False          False        False   
1     False  False          False          False        False   
2     False  False          False          False        False   
3     False  False          False          False        False   
4     False  False          False          False        False   
...     ...    ...            ...            ...          ...   
7995  False  False          False          False        False   
7996  False  False          False          False        False   
7997  False  False          False          False        False   
7998  False  False          False          False        False   
7999  False  False          False          False        False   

      distances__startDistanceFromShoreKm  distances__endDistanceFromShoreKm  \
0                                   False                              False   
1                           

In [4]:
# replace NaN flag with 'MISSING'; not dropping because it's may be a useful feature
df['vessel__flag'].fillna('MISSING', inplace=True)
df['encounter__vessel__flag'].fillna('MISSING', inplace=True)

/var/folders/q0/l7dmy8x535g5clgjv2m_0pnm0000gn/T/ipykernel_94631/2848861823.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['vessel__flag'].fillna('MISSING', inplace=True)
/var/folders/q0/l7dmy8x535g5clgjv2m_0pnm0000gn/T/ipykernel_94631/2848861823.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values al

In [5]:
# to double confirm that there is no missing flag now
df.isna().sum()

start                                  0
end                                    0
position__lat                          0
position__lon                          0
boundingBox                            0
distances__startDistanceFromShoreKm    0
distances__endDistanceFromShoreKm      0
distances__startDistanceFromPortKm     0
distances__endDistanceFromPortKm       0
encounter__medianDistanceKilometers    0
encounter__medianSpeedKnots            0
encounter__potentialRisk               0
vessel__id                             0
vessel__ssvid                          0
vessel__flag                           0
vessel__type                           0
encounter__vessel__id                  0
encounter__vessel__ssvid               0
encounter__vessel__flag                0
encounter__vessel__type                0
duration_hours                         0
dtype: int64